In [1]:
from aocd.models import Puzzle
from pprint import pprint
import random
from copy import deepcopy

In [2]:
puzzle_input = Puzzle(2021, 12).input_data.split("\n")
small_test_input = [
	"start-A",
	"start-b",
	"A-c",
	"A-b",
	"b-d",
	"A-end",
	"b-end"
]
medium_test_input = [
	"dc-end",
	"HN-start",
	"start-kj",
	"dc-start",
	"dc-HN",
	"LN-dc",
	"HN-end",
	"kj-sa",
	"kj-HN",
	"kj-dc",
]
large_test_input = [
	"fs-end",
	"he-DX",
	"fs-he",
	"start-DX",
	"pj-DX",
	"end-zg",
	"zg-sl",
	"zg-pj",
	"pj-he",
	"RW-he",
	"fs-DX",
	"pj-RW",
	"zg-RW",
	"start-pj",
	"he-WI",
	"zg-he",
	"pj-fs",
	"start-RW",
]

In [3]:
def generate_connection_directory(connection_list):
	connection_map = {}
	for connection in connection_list:
		connection_start = connection.split("-")[0]
		connection_end = connection.split("-")[1]
		
		if connection_start not in connection_map:
			connection_map[connection_start] = [connection_end]
		if connection_start in connection_map:
			if connection_end not in connection_map[connection_start]:
				connection_map[connection_start].append(connection_end)

		if connection_end not in connection_map:
			connection_map[connection_end] = [connection_start]
		if connection_end in connection_map:
			if connection_start not in connection_map[connection_end]:
				connection_map[connection_end].append(connection_start)

	return connection_map

In [124]:
def generate_paths(input, part):
	
	connection_dict = deepcopy(input)
	valid_paths = []
	paths_stack = []
	count = 0

	if part == "a":
		# since small caves can only be visited at most once, and big caves any number of times
		# disregard any small caves which only have one connection to another small cave,
		# as these ones cant be visited in the first place.
		connection_dict = {x:connection_dict[x]
								for x,connection_dict[x] in connection_dict.items()
								if len(connection_dict[x]) > 1 or
								connection_dict[x][0] == connection_dict[x][0].upper()}
	
	current_path = "start,"
	paths_stack.append("start")
	small_cave_visited_twice = False
	backtrack_neighbors = None

	paths_stack_previous_step = None

	while paths_stack and count < 1000:
		current_cave = paths_stack[-1]

		print(paths_stack)
		print(f"{current_cave}'s backtrack neighbors: {backtrack_neighbors}")

		if part == "a":
			# if not [x for x in valid_paths if x.startswith(",".join(paths_stack))]:
			# 	backtrack_neighbors = None

			neighbors = backtrack_neighbors if backtrack_neighbors is not None else connection_dict[current_cave]

			possible_next_caves = [x for x in neighbors
									if x in connection_dict 
									and (x == x.upper() or x not in current_path)
									and current_path + f"{x}" not in valid_paths]

			# no path in valid paths may startwith() the current path
			for cave in possible_next_caves:
				if [x for x in valid_paths if x.startswith(current_path + cave)]:
					possible_next_caves.remove(cave)


					
		elif part == "b":
			possible_next_caves = [x for x in connection_dict[current_cave] 
									if (x == x.upper() or (x != "start" and (x not in current_path or not small_cave_visited_twice)))
									or x == "end"]
		
		if not possible_next_caves:
			print("cul-de-sac")
			current_cave = paths_stack.pop()
			if not paths_stack: break
			previous_cave = paths_stack[-1]
			backtrack_neighbors = [cave for cave in connection_dict[previous_cave] if (cave != current_cave)]
			current_path = ",".join(current_path.split(",")[:-2]) + ","

		else:
			next_cave = possible_next_caves[0]
			current_path += f"{next_cave},"
			paths_stack.append(next_cave)
			if next_cave == "end":
				current_path = current_path[:-1]  # remove trailing comma
				valid_paths.append(current_path)
				print(f"Added {current_path} to valid paths")
				paths_stack.pop()  # remove "end" from stack and continue search
			else:
				# no longer backtracking
				backtrack_neighbors = None

		count += 1

	pprint(valid_paths)
	print(len(valid_paths))


In [121]:
# The difference between this function and generate_path() is that this function allows for a *single* small
# cave (provided it is not the start and end cave) to be visitied twice.

def generate_path_2(input_directory):
	new_input_directory = deepcopy(input_directory)

	path = "start,"
	current_cave = "start"
	restart = False
	small_cave_visited_twice = False

	while "end" not in path:
		possible_next_caves = [x for x in new_input_directory[current_cave] 
								if (x == x.upper() or (x != "start" and (x not in path or not small_cave_visited_twice)))
								or x == "end"]
		# print(current_cave, possible_next_caves)
		if not possible_next_caves:
			# all possible caves are small caves which have already been explored. You have reached a cul-de-sac. Restart path finding process.
			# print("Cul de sac. Restarting...")
			restart = True
		else:
			next_cave = random.choice(possible_next_caves)
			if next_cave == next_cave.lower() and next_cave in path:
				small_cave_visited_twice = True
			path += f"{next_cave},"
			current_cave = next_cave

		if restart:
			path = "start,"
			current_cave = "start"
			restart = False
			small_cave_visited_twice = False
			continue


	return path[:-1]  # remove comma behind "end"

In [122]:
small_test_connection_directory = generate_connection_directory(small_test_input)
medium_test_connection_directory = generate_connection_directory(medium_test_input)
large_test_connection_directory = generate_connection_directory(large_test_input)
puzzle_input_connection_directory = generate_connection_directory(puzzle_input)

In [126]:
# part a
generate_paths(medium_test_connection_directory, part="a")

['start']
start's backtrack neighbors: None
['start', 'HN']
HN's backtrack neighbors: None
['start', 'HN', 'dc']
dc's backtrack neighbors: None
Added start,HN,dc,end to valid paths
['start', 'HN', 'dc']
dc's backtrack neighbors: None
['start', 'HN', 'dc', 'HN']
HN's backtrack neighbors: None
['start', 'HN', 'dc', 'HN', 'kj']
kj's backtrack neighbors: None
['start', 'HN', 'dc', 'HN', 'kj', 'HN']
HN's backtrack neighbors: None
cul-de-sac
['start', 'HN', 'dc', 'HN', 'kj']
kj's backtrack neighbors: ['start', 'sa', 'dc']
cul-de-sac
['start', 'HN', 'dc', 'HN']
HN's backtrack neighbors: ['start', 'dc', 'end']
cul-de-sac
['start', 'HN', 'dc']
dc's backtrack neighbors: ['end', 'start', 'LN', 'kj']
['start', 'HN', 'dc', 'kj']
kj's backtrack neighbors: None
['start', 'HN', 'dc', 'kj', 'HN']
HN's backtrack neighbors: None
Added start,HN,dc,kj,HN,end to valid paths
['start', 'HN', 'dc', 'kj', 'HN']
HN's backtrack neighbors: None
cul-de-sac
['start', 'HN', 'dc', 'kj']
kj's backtrack neighbors: ['sta

In [44]:
# part b


In [59]:
"abcd" in ["abcde"]

False